In [3]:
# Filtro de Calidad para las Imágenes

import cv2
import numpy as np
from PIL import Image, ImageStat

# Parámetros mínimos
MIN_RESOLUTION = (224, 224)  # Ancho x Alto
MIN_BRIGHTNESS = 50  # Umbral mínimo de luminosidad
MIN_LAPLACIAN_VARIANCE = 100  # Umbral para enfoque

def check_image_quality(image_path):
    """
    Filtrar imágenes por resolución, tamaño, enfoque y luminosidad.
    """
    try:
        # Cargar la imagen
        img = Image.open(image_path)
        img_cv = cv2.imread(image_path)

        # Comprobar resolución y tamaño
        if img.size < MIN_RESOLUTION:
            print(f"Imagen descartada por tamaño: {image_path}")
            return False

        # Comprobar luminosidad
        brightness = np.mean(img_cv)
        if brightness < MIN_BRIGHTNESS:
            print(f"Imagen descartada por baja luminosidad: {image_path}")
            return False

        # Comprobar enfoque
        laplacian_var = cv2.Laplacian(cv2.cvtColor(img_cv, cv2.COLOR_BGR2GRAY), cv2.CV_64F).var()
        if laplacian_var < MIN_LAPLACIAN_VARIANCE:
            print(f"Imagen descartada por desenfoque: {image_path}")
            return False

        return True  # Imagen válida
    except Exception as e:
        print(f"Error procesando {image_path}: {e}")
        return False

image_path = "/workspace/pretrainfoodclassificationwidget/classified/no"
check_image_quality(image_path)


ImportError: libGL.so.1: cannot open shared object file: No such file or directory